### Sorting, abstract class

In [45]:
import numpy as np
import functools
import copy
import operator


class Sorting(object):
    """排序算法抽象类
    
    - 通用功能：交换和移动
        + swap()
        + move()
    - 数据生成： random, almost sorted, reversed, few unique
    - 性能分析： time: #swap, #move; space: 
    """
    
    def __init__(self):
        
        # run & performance info
        self.origin_L = None
        self.L = None
        self.N = None
        
        self.count_call_dict = {}
        
        self.alg_cmp = self.count_call(self.alg_cmp)
        self.swap = self.count_call(self.swap)
        self.copy_to = self.count_call(self.copy_to)
    
    @staticmethod
    def generate_L(N=30, few_unique_nvalue=5, seed=123456):
        np.random.seed(seed)
        return {
            'L_random': list(np.random.choice(range(N), size=N, replace=False)),
            'L_sorted': list(range(N)),
            'L_reversed': list(range(N, 0, -1)),
            'L_few_unique': list(np.random.choice(range(few_unique_nvalue), size=N, replace=True)),
        }
    
    # ------------ sort -----------------------------
    def sort(self, L_):
        L = copy.deepcopy(L_)
        if True:
            self.origin_L = copy.deepcopy(L)
            self.L = L
            self.N = len(L)
            
        self._sort(self.L)
        
        if True:
            Sorting.check_sorted(self.L)
            self.perfromance_summary()
        
    def _sort(self, L):
        raise NotImplementedError
        
    # ------------ utils && performance analysis -----------------------------    
    def count_call(self, func):
        self.count_call_dict[func.__name__] = 0
        @functools.wraps(func)
        def wrapper(*args, **kw):
            self.count_call_dict[func.__name__] += 1
            return func(*args, **kw)
        return wrapper
    
    def alg_cmp(self, a, op, b):
        return op(a, b)
    
    def swap(self, L, i, j):
        L[i], L[j] = L[j], L[i]
        
    def copy_to(self, L, i, value):
        # L[i] = value   # value should be object that pass by value
        L[i] = copy.deepcopy(value)   # just in case of
            
    def perfromance_summary(self):
        print(self.origin_L)
        print('inversion number:', Sorting.get_inversion_number(self.origin_L))
        print(self.L, Sorting.check_sorted(self.L))
        print('------------------------')
        print('N =', self.N)
        print('========= Theretical Analysis =========')
        for (txt, f) in self.theretical_complexity():
            print(txt, '-->', str(f(self.N)))
        print('------------------------')
        print('time complexity: ', self.count_call_dict)
        print('space complexity:', )
        
    def theretical_complexity(self):
        raise NotImplementedError
        
    @staticmethod
    def get_inversion_number(L):
        n = len(L)
        return sum(L[i] > L[j] for i in range(n) for j in range(i+1, n))
    
    @staticmethod
    def check_sorted(L):
        return all(L[i] <= L[i+1] for i in range(len(L) - 1))

In [46]:
data = Sorting.generate_L(N=20)
L1 = data['L_random']
L2 = data['L_few_unique']

### Bubble Sort

In [47]:
class BubbleSort(Sorting):
    """冒泡排序"""
    
    def _sort(self, L):
        n = len(L)
        for i in range(n):
            for j in range(n-1, i, -1):
                if self.alg_cmp(L[j-1], operator.__gt__, L[j]):
                    self.swap(L, j-1, j)
                    
    def theretical_complexity(self):
        return [
            ('#cmp: N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#swap  bad-case reversed: N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#swap  good-case sorted: 0', lambda N: 0),
            ('#space: 0', lambda N: 0)
        ]

In [48]:
m_bb = BubbleSort()
m_bb.sort(L1)

[19, 13, 12, 9, 5, 6, 2, 3, 18, 8, 4, 7, 0, 14, 16, 15, 11, 17, 10, 1]
inversion number: 101
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case reversed: N(N-1)/2 --> 190.0
#swap  good-case sorted: 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 101, 'copy_to': 0}
space complexity:


In [49]:
m_bb2 = BubbleSort()
m_bb2.sort(L2)

[4, 4, 4, 4, 3, 1, 4, 3, 3, 0, 4, 2, 2, 0, 3, 4, 2, 1, 2, 4]
inversion number: 100
[0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case reversed: N(N-1)/2 --> 190.0
#swap  good-case sorted: 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 100, 'copy_to': 0}
space complexity:


### Selection Sort

In [50]:
class SelectionSort(Sorting):
    """选择排序"""
    
    def _sort(self, L):
        n = len(L)
        for i in range(n):
            min_idx = i
            for j in range(i+1, n):
                if self.alg_cmp(L[j], operator.__lt__, L[min_idx]):
                    min_idx = j
            if min_idx > i:
                self.swap(L, i, min_idx)  # if min_idx > i, 这个比较算不算在时间复杂度内？
                    
    def theretical_complexity(self):
        return [
            ('#cmp: N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#swap  bad-case(most case, swap for all i): N', lambda N: N),
            ('#swap  good-case(sorted): 0', lambda N: 0),
            ('#space: 0', lambda N: 0)
        ]

In [51]:
m_selection = SelectionSort()
m_selection.sort(L1)

[19, 13, 12, 9, 5, 6, 2, 3, 18, 8, 4, 7, 0, 14, 16, 15, 11, 17, 10, 1]
inversion number: 101
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case(most case, swap for all i): N --> 20
#swap  good-case(sorted): 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 17, 'copy_to': 0}
space complexity:


In [52]:
m_selection2 = SelectionSort()
m_selection2.sort(L2)

[4, 4, 4, 4, 3, 1, 4, 3, 3, 0, 4, 2, 2, 0, 3, 4, 2, 1, 2, 4]
inversion number: 100
[0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case(most case, swap for all i): N --> 20
#swap  good-case(sorted): 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 11, 'copy_to': 0}
space complexity:


In [53]:
m_selection3 = SelectionSort()
m_selection3.sort([0, 0, 0, 1, 1, 1])

[0, 0, 0, 1, 1, 1]
inversion number: 0
[0, 0, 0, 1, 1, 1] True
------------------------
N = 6
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 15.0
#swap  bad-case(most case, swap for all i): N --> 6
#swap  good-case(sorted): 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 15, 'swap': 0, 'copy_to': 0}
space complexity:


### Insertion Sort

In [69]:
class InsertionSort(Sorting):
    """插入排序"""
    
    def _sort(self, L):
        n = len(L)
        L.append(None)
        TMP = n
        
        # [  0, x, x, x, x, x, x, x,   i-1,   i,  ]
        # [j=0 <------------j------- j=i-1,   i,  ]
        # [                 k >>>>>>>>>>>>>>>>|,  ]
        for i in range(n):
            for j in reversed(range(i)):
                if self.alg_cmp(L[i], operator.__ge__, L[j]):  # L[i] 放到 j 后面
                    self.copy_to(L, TMP, L[i])
                    # tmp_storage = copy.deepcopy(L[i])  # todo: copy_to
                    for k in range(i, j+1, -1):
                        self.copy_to(L, k, L[k-1])
                        #L[k] = copy.deepcopy(L[k-1])
                    self.copy_to(L, j+1, L[TMP])
                    #L[j+1] = copy.deepcopy(tmp_storage)
                    break
                    
            # here j == 0, if not break
            if L[i] < L[0]:
                self.copy_to(L, TMP, L[i])
                # tmp_storage = copy.deepcopy(L[i])
                for k in range(i, 0, -1):
                    self.copy_to(L, k, L[k-1])
                    #L[k] = copy.deepcopy(L[k-1])
                self.copy_to(L, 0, L[TMP])
                #L[0] = copy.deepcopy(tmp_storage)
        del L[TMP]
                    
    def theretical_complexity(self):
        return [
            ('#cmp bad-case(reversed): 1 + ... + N-1 = N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#cmp good-case(sorted): N-1', lambda N: N-1),
            ('#copy  bad-case(reversed): 3 + ... + N+1 = (N+4)(N-1) / 2', lambda N: (N+4) * (N-1) / 2),
            ('#copy  good-case(sorted): 2(N-1)', lambda N: 2 * (N-1)),
            ('#space: 1', lambda N: 1)
        ]

In [70]:
m_insert = InsertionSort()
m_insert.sort(L1)

[19, 13, 12, 9, 5, 6, 2, 3, 18, 8, 4, 7, 0, 14, 16, 15, 11, 17, 10, 1]
inversion number: 101
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp bad-case(reversed): 1 + ... + N-1 = N(N-1)/2 --> 190.0
#cmp good-case(sorted): N-1 --> 19
#copy  bad-case(reversed): 3 + ... + N+1 = (N+4)(N-1) / 2 --> 228.0
#copy  good-case(sorted): 2(N-1) --> 38
#space: 1 --> 1
------------------------
time complexity:  {'alg_cmp': 114, 'swap': 0, 'copy_to': 139}
space complexity:


In [71]:
m_insert2 = InsertionSort()
m_insert2.sort(L2)

[4, 4, 4, 4, 3, 1, 4, 3, 3, 0, 4, 2, 2, 0, 3, 4, 2, 1, 2, 4]
inversion number: 100
[0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp bad-case(reversed): 1 + ... + N-1 = N(N-1)/2 --> 190.0
#cmp good-case(sorted): N-1 --> 19
#copy  bad-case(reversed): 3 + ... + N+1 = (N+4)(N-1) / 2 --> 228.0
#copy  good-case(sorted): 2(N-1) --> 38
#space: 1 --> 1
------------------------
time complexity:  {'alg_cmp': 116, 'swap': 0, 'copy_to': 138}
space complexity:


In [74]:
m_insert3 = InsertionSort()
m_insert3.sort([0, 1, 2, 3, 4, 5, 1])

[0, 1, 2, 3, 4, 5, 1]
inversion number: 4
[0, 1, 1, 2, 3, 4, 5] True
------------------------
N = 7
========= Theretical Analysis =========
#cmp bad-case(reversed): 1 + ... + N-1 = N(N-1)/2 --> 21.0
#cmp good-case(sorted): N-1 --> 6
#copy  bad-case(reversed): 3 + ... + N+1 = (N+4)(N-1) / 2 --> 33.0
#copy  good-case(sorted): 2(N-1) --> 12
#space: 1 --> 1
------------------------
time complexity:  {'alg_cmp': 10, 'swap': 0, 'copy_to': 16}
space complexity:


## Python笔记：关于 funcion 和 bound method

> https://stackoverflow.com/questions/7891277/why-does-setattr-fail-on-a-bound-method


### 遇到的问题：试图对成员函数setattr时会抛出错误

AttributeError: 'method' object has no attribute 'foo'

### 原理：function 与 bound method 的本质是什么？

function object 是某种“真实存在的”对象，可以设置任意attr，它有一个`__dict__`属性来存储所有的attr    

而与特定实例绑定的 bound method 不同，它没有具体的函数实体。

class中的def会创建一个（属于cls命名空间中的）funcion object，通过`cls.m`获得。   
而与具体实例绑定的 bound method 不存储函数实体，它只是是记录两个对象的引用：

- 其宿主 `instance.__self__`
- 定义在类中的函数实体 `instance.m.__func__` (is `cls.m`)

当bound method 被调用时，实际执行的是`cls.m(instance.__self__, ...)`   
该类所有实例的method，其函数实体`instance.m.__func__` 都指向同一个对象，即类创建时创建的 function object `cls.m`

> 回忆quantx是怎么在R语言中实现“类”的？

### 回到刚才的问题

commit e785ee8 实际上是将属性设置在了cls.m上（定义在class namespace中，是函数对象），是错误的。

```
def dec(func):
    def wrapper(*arg, **kw):
        func.xx = xx              # ✔️ 不会报错，但注意，属性是设置在了 KLS.func上
        return func(*arg, **kw)
    return wrapper

class KLS:
    @dec
    def foo:
        setattr(self.bar, 'xx', xx)   # ❌ bounded method 不是函数对象，不可以设置属性
        setattr(self.foo, 'xx, xx)  # ❌原因同上

ins = KLS()
ins.foo.xx = xx   # ❌ 原因同上
```

In [26]:
m_bb.swap is m_bb2.swap   # different object, since it is bound method of two different istance

False

In [24]:
m_bb.swap.__func__ is m_bb2.swap.__func__  # same object ...

True

In [29]:
m_bb.swap.__func__ is BubbleSort.swap    # ... since they are all ref to cls.funcion

True

In [31]:
m_bb.swap is BubbleSort.swap   # bound method and cls.funcion is not same thing !

False

In [18]:
BubbleSort.swap  # it is function object !

<function __main__.Sorting.swap(self, L, i, j)>

In [19]:
m_bb.swap  # it is bound method

<bound method Sorting.swap of <__main__.BubbleSort object at 0x1057068d0>>

In [20]:
m_bb2.swap  # bound method of another istance

<bound method Sorting.swap of <__main__.BubbleSort object at 0x105706390>>

### 那么怎么实现“成员方法调用次数计数”这个需求呢？我自己想的一个实现：decorator最原始的语法

> https://medium.com/@vadimpushtaev/decorator-inside-python-class-1e74d23107f6

1. 直接在成员方法的`def`上加`@dec`并不work
2. staticmethod也不行，因为 @staticmethod 对于inside class / outside class 的behavior不同

In [63]:
class A(object):
    def __init__(self):
        print('calling __init__')
        self.count_call_dict = {}
        
        self.a = self.count_call(self.a)
        self.b = self.count_call(self.b)
        
    def a(self):
        print('aaaaaaaaaaa')
    
    def b(self):
        print('bbbbbbbbbbb')
        
    def count_call(self, func):
        print('Calling count_call')
        self.count_call_dict[func.__name__] = 0
        def wrapper(*args, **kw):
            print('func name is', func.__name__)
            self.count_call_dict[func.__name__] += 1
            return func(*args, **kw)
        return wrapper

In [69]:
foooo = A()
b___ = A()

foooo.a(); foooo.a(); foooo.a()   # foooo.a = 3
foooo.b(); foooo.b()              # foooo.b = 2
b___.a()
foooo.a()                         # foooo.a = 4
b___.b(); b___.b()

print(foooo.count_call_dict)
print(b___.count_call_dict)

calling __init__
Calling count_call
Calling count_call
calling __init__
Calling count_call
Calling count_call
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is b
bbbbbbbbbbb
func name is b
bbbbbbbbbbb
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is b
bbbbbbbbbbb
func name is b
bbbbbbbbbbb
{'a': 4, 'b': 2}
{'a': 1, 'b': 2}


## Python Notes: pass by assignment

> [python document](https://docs.python.org/3/faq/programming.html#how-do-i-write-a-function-with-output-parameters-call-by-reference)    
> "Arguments are **passed by assignment** in Python."

> [A blog](https://robertheaton.com/2014/02/09/pythons-pass-by-object-reference-as-explained-by-philip-k-dick/)    
> What does it means by saying **"Object references are passed by value"**?

> https://nedbatchelder.com/text/names1.html
